# PBR 로 적정 주가 산출 후 현재 주가가 적정 주가보다 20% 이상 낮을 경우 매수

In [12]:
#PER 
#적정 PER -> 낮을수록 좋다. 10이하 6~4정도면 좋음
#5년 평균 PER * EPS = 적정주가 
#적정주가 /1.2 OR 0.8 = 20% 낮은 가격 ->매수 
#적정주가 /현재주가 = 1 -> 현재주가가 적정하다 
#0.96 -> 4퍼센트정도 과대평가 
#1.04 -> 4%정도 낮은 주가 

#PBR
#적정 PBR -> 낮을 수록 좋다. 1이면 현재 적정주가라는 것, 그 이하라면 좋음 
#가장 최근 PBS (PBS는 그간 쌓인 자산이니 가장 최근 값을 보면 된다)
#5년 평균 PBR * BPS = 적정주가 * 0.8 -> 매수 

#미래 주가 예측 
#미래 적정 주가 = 미래의 EPS(애널리스트 추정치) * 적정PER(5년간의 평균)

In [13]:
# 3가지 주가예측 방법으로 3년간의 주가를 체크해보고 주가 흐름과 가장 근접한 방법을 선택함 

#testCode
#url = 'https://finance.naver.com/item/main.nhn?code=005930'
#table_df_list = pd.read_html(url, encoding='euc-kr')    # 한글이 깨짐. utf-8도 깨짐. 그래서 'euc-kr'로 설정함  
#table_df = table_df_list[3]

#eps_2018 = table_df.iloc[9,1]
#per_2018 = table_df.iloc[10,1]
#bps_2018 = table_df.iloc[11,1]
#pbr_2018 = table_df.iloc[12,1]
#roe_2018 = table_df.iloc[5,1]

#eps_2019 = table_df.iloc[9,2]
#per_2019 = table_df.iloc[10,2]
#bps_2019 = table_df.iloc[11,2]
#pbr_2019 = table_df.iloc[12,2]
#roe_2019 = table_df.iloc[5,2]

#eps_2021 = table_df.iloc[9,-2]
#per_2021 = table_df.iloc[10,-2]
#bps_2021 = table_df.iloc[11,-2]
#pbr_2021 = table_df.iloc[12,-2]
#roe_2021 = table_df.iloc[5,-2]

#eps_2020 = table_df.iloc[9,3]
#per_2020 = table_df.iloc[10,3]
#bps_2020 = table_df.iloc[11,3]
#pbr_2020 = table_df.iloc[12,3]
#roe_2020 = table_df.iloc[5,3]

#eps2018 = eps_2018 * roe_2018 * 100
#titrationPrice_2018 = per_2018 * eps_2018
#pbr2018 =  pbr_2018 * bps_2018
#print(eps2018)
#print(titrationPrice_2018)
#print(pbr2018)

#eps2019 = eps_2019 * roe_2019 * 100
#titrationPrice_2019 = per_2019 * eps_2019
#pbr2019 =  pbr_2019 * bps_2019
#print(eps2019)
#print(titrationPrice_2019)
#print(pbr2019)

#eps2020 = eps_2020 * roe_2020 * 100
#titrationPrice_2020 = per_2020 * eps_2020
#pbr2020 =  pbr_2020 * bps_2020
#print(eps2020)
#print(titrationPrice_2020)
#print(pbr2020)

#eps2021 = eps_2021 * roe_2021 * 100
#titrationPrice_2021 = per_2021 * eps_2021
#pbr2021 = pbr_2021 * bps_2021
#print(eps2021)
#print(titrationPrice_2021)
#print(pbr2021)

In [15]:
import urllib.parse
from bs4 import BeautifulSoup 
import requests 
import pandas as pd
import time
import datetime
import math

MARKET_CODE_DICT = {
    
    'kospi': 'stockMkt',
    'kosdaq': 'kosdaqMkt',
    'konex': 'konexMkt'
    
}
DOWNLOAD_URL = 'https://kind.krx.co.kr/corpgeneral/corpList.do'
naver_stock = 'https://finance.naver.com/item/main.nhn?code='

In [16]:
#종목코드 6자리 미만시 사용할 함수 
def zeroFill(columnValue):
    
    columnValue = str(columnValue)
    #앞을 0으로 채워라 제로필
    outValue = columnValue.zfill(6)  

    return outValue

In [17]:
#코스피 종목 불러올 함수 설정 
def get_stock_codes(market=None, delisted=False):
    
    params = {'method': 'download'}
    
    if market.lower() in MARKET_CODE_DICT:
        
        ## marketType 키 추가
        params['marketType'] = MARKET_CODE_DICT[market]
        #print(market.lower()+" market key is exist")
        
    else:
        
        print("invalid market")

    params_string = urllib.parse.urlencode(params)
    request_url = DOWNLOAD_URL+"?"+params_string
    df = pd.read_html(request_url)[0]
    df["종목코드"] = df.종목코드.apply(zeroFill)
 
    return df

In [18]:
#종목 테이블에서 종목코드(100개) 가져오는 함수 
def get_code(market):
    
    codeList = []
    stocks = get_stock_codes(market)
    
    for i in range(len(stocks)):  #해당 마켓의 전체 종목 가지고오기 
        
        stock_code = stocks.iloc[i]['종목코드']
        codeList.append(stock_code)
        #stock_name = stocks.iloc[0]['회사명']
        #print(stock_code)
        
    return codeList

In [19]:
#주가정보 테이블 생성 함수 
def stock_info(stockCode):
    
    try:
    
        url = naver_stock + stockCode #종목코드 url에 추가 
        table_df_list = pd.read_html(url, encoding='euc-kr')    # 한글이 깨짐. utf-8도 깨짐. 그래서 'euc-kr'로 설정함  
        table_df = table_df_list[3] #종목 정보가 담겨있는 테이블 추출 

        return table_df
    
    except:
        
        return None

In [20]:
#현재년도의 bps와 per 값 구하해 매입목표가격 정의 
def get_target_price(stockInfo): 
    
    try:
        
        bps_this_year = stockInfo.iloc[11,-2] #bps값 정의
        pbr_this_year = stockInfo.iloc[12,-2] #pbr값 정의
        just_price = pbr_this_year * bps_this_year #적정가격 도출 
        target_price = math.ceil(just_price * 0.8) #적정가격보다 20% 더 저렴한 가격 도출 -> 소수점 올림 (절대값으롳 처리했습니다) 
        
    except : #가끔 bps가 존재하지 않는 경우 발생 -> 그럴 경우 말이 안되게 높은 금액으로 설정해 잘못된 매수의견 출력을 방지함 
        
        target_price = 10
    
    return target_price

In [21]:
#주식 현재 가격 받아오는 함수 
def get_price(stockCode):
    
    url = naver_stock + stockCode #종목 코드 url에 추가 
    result = requests.get(url)
    html = BeautifulSoup(result.content, "html.parser") #태그에서 현재가격 가지고오기 
    today = html.find("p", {"class":"no_today"}) 
    real_time_tag = today.find("span", {"class":"blind"})
    real_time_price = int(real_time_tag.text.replace(',',"")) #컴마 제거 후 인트형으로 변환 
    
    return real_time_price

In [22]:
#주식 종목명 가지져오는 함수 
def get_name(market , i):
    
    stocks = get_stock_codes(market)
    stock_name = stocks.iloc[i]['회사명']
        
    return stock_name

In [23]:
#매수의견 받아오기 
def investment_opinion(stockCode):
    
    try:
    
        url = naver_stock + stockCode
        stock_info_list = pd.read_html(url, encoding='euc-kr')
        stock_info = stock_info_list[7]
        table = stock_info[1][0]
        investment_opinion = table.split("l")[0].strip()[-2:] #테이블에서 투자자 의견 가지고오기
        
        if investment_opinion == '/A' : 
            investment_opinion = None
        
        #투자자 의견이 매수/강력매수인 경우에만 리턴     
        if (investment_opinion =='중립') or (investment_opinion =='매수') or (investment_opinion == '강력매수')  or (investment_opinion == None): 
            return investment_opinion
   
    except : 
        return -1

In [24]:
#요일 계산 함수 (주말에는 프로그램이 돌지 않는다)
def get_days():
    
    result = datetime.datetime.today().weekday()
    
    if (result == 5) or (result == 6) :
        
        return False
    
    else : 
        
        return True

In [25]:
#시간 가지고 오기 (09:00 ~ 15:30 까지만 자동으로 돌 수 있도록 )
def get_time():
    
    now = datetime.datetime.now()
    time = now.strftime("%H")
    minute = now.strftime("%M")
    result = "{}{}".format(time,minute)
    
    return result 

In [26]:
#사용자가 원할 때 사용할 수 있는 매수/매도 추천 함수 
#매수 추천 결과가 N이 압도적으로 많지만 낮은 빈도로 Y(매수의견) 을 주는것을 확인했습니다!
#매수 추천이 y인 것만 출력할까 했는데 추천 종목 뿐만 아니라 사용자가 자신이 선택한 마켓 (코스피 등)의 여러 종목을 검색해보고  
#네이버 투자자 의견 / 제가 짠 코드의 매수 추천 의견을 동시에 검토해보고 결정할 수 있도록 하면 좋을 것 같아 Y/N모두 출력하게 하였습니다.

def stock_advisor(market):
        
    codeList = get_code(market) #사용자가 입력한 마켓 ( 코스피 / 코스닥 / 코넥스 ) 의 종목코드 전부 가지고오기 

    for i in range(len(codeList)): #종목리스트의 길이만큼 포문 

        stockCode = codeList[i]

        stockInfo = stock_info(stockCode) #종목코드 추출

        if len(stockInfo) > 0  : #테이블 추출에 성공했다면 아래 로직 수행 

            stock_name = get_name(market,i) #종목명 
            target_price = get_target_price(stockInfo) #목표가격
            real_time_price = get_price(stockCode) #현재가격 
            investment = investment_opinion(stockCode) #네이버투자자의견 (매수/강력매수/none인 경우만 )

            result = "" #매수/매도 결정을 담아줄 변수 선언 

            if investment != -1 : #투자자 의견이 매수 / 강력매수 / none 인 경우에만 가격 확인  

                if(real_time_price < target_price): #현재 가격이 우리의 목표가격 ( 우리가 구한 적정가격보다 20% 이상 저렴할 경우 y 아니라면 n)

                    result = "Y"

                else : 
                    
                    result = "N"
                    
                #출력할 문자열 정의 
                pString = "{}. 종목명 : {} / 종목코드 : {} / 목표매수가격 : {} / 현재가격 : {} / 네이버투자의견 : {} / 매수추천 : {}".format(i , stock_name , codeList[i] , target_price , real_time_price , investment , result)

                print(pString) #최종결과 출력 

In [27]:
#자동화를 위한 코드 (매주 월요일에 돌릴 수 있도록 ) 자동화 코드를 추가했습니다
def stock_advisor2(market):
    
    while get_days() : #평일동안 반복문 돌기 
        
        time = get_time() #현재 시간 가지고오기 
        
        if time == "9000" : # 평일 09:00이 되면 프로그램 시작 
            
            while True: #무한반복 
                
                if time == "1530" : #장 마감시 break
                    
                    break
        
                codeList = get_code(market) #사용자가 입력한 마켓 ( 코스피 / 코스닥 / 코넥스 ) 의 종목코드 전부 가지고오기 

                for i in range(len(codeList)): #종목리스트의 길이만큼 포문 
                    
                    stockCode = codeList[i]

                    stockInfo = stock_info(stockCode) #종목코드 추출

                    if len(stockInfo) > 0  : #테이블 추출에 성공했다면 아래 로직 수행 

                        stock_name = get_name(market,i) #종목명 
                        target_price = get_target_price(stockInfo) #목표가격
                        real_time_price = get_price(stockCode) #현재가격 
                        investment = investment_opinion(stockCode) #투자자의견 

                        result = "" #매수/매도 결정을 담아줄 변수 선언 

                        if investment : #투자자 의견이 매수 / 강력매수 / 없음 인 경우에만 가격 확인  

                            if(real_time_price < target_price): #현재 가격이 우리의 목표가격 ( 우리가 구한 적정가격보다 20% 이상 저렴할 경우 y 아니라면 n)

                                result = "Y"

                            else : 
                                
                                result = "N"
                                
                            #출력할 문자열 정의 
                            pString = "{}. 종목명 : {} / 종목코드 : {} / 적정가격 : {} / 현재가격 : {} / 투자의견 : {} / 매수추천 : {}".format(i , stock_name , codeList[i] , target_price , real_time_price , investment , result)

                            print(pString) #최종결과 출력 

In [29]:
stock_advisor('kospi') 

0. 종목명 : DRB동일 / 종목코드 : 004840 / 목표매수가격 : 5343 / 현재가격 : 8480 / 네이버투자의견 : None / 매수추천 : N
1. 종목명 : DSR / 종목코드 : 155660 / 목표매수가격 : 4564 / 현재가격 : 7060 / 네이버투자의견 : None / 매수추천 : N
2. 종목명 : GS글로벌 / 종목코드 : 001250 / 목표매수가격 : 1997 / 현재가격 : 3185 / 네이버투자의견 : None / 매수추천 : N
3. 종목명 : HDC현대산업개발 / 종목코드 : 294870 / 목표매수가격 : 23350 / 현재가격 : 30000 / 네이버투자의견 : 매수 / 매수추천 : N
4. 종목명 : KEC / 종목코드 : 092220 / 목표매수가격 : 1857 / 현재가격 : 2570 / 네이버투자의견 : None / 매수추천 : N
5. 종목명 : KG동부제철 / 종목코드 : 016380 / 목표매수가격 : 12279 / 현재가격 : 19950 / 네이버투자의견 : None / 매수추천 : N
6. 종목명 : KG케미칼 / 종목코드 : 001390 / 목표매수가격 : 21585 / 현재가격 : 29400 / 네이버투자의견 : None / 매수추천 : N
7. 종목명 : KTis / 종목코드 : 058860 / 목표매수가격 : 2392 / 현재가격 : 3290 / 네이버투자의견 : None / 매수추천 : N
8. 종목명 : LG이노텍 / 종목코드 : 011070 / 목표매수가격 : 164724 / 현재가격 : 211000 / 네이버투자의견 : 매수 / 매수추천 : N
9. 종목명 : LG전자 / 종목코드 : 066570 / 목표매수가격 : 119906 / 현재가격 : 151500 / 네이버투자의견 : 매수 / 매수추천 : N
10. 종목명 : LG헬로비전 / 종목코드 : 037560 / 목표매수가격 : 3915 / 현재가격 : 7890 / 네이버투자의견 : 중립 / 매수추천 : N
11. 종목명 : OCI 